In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Set the path to your labeled dataset folders
malicious_dir = 'QR codes\Malicious\malicious'
non_malicious_dir = r'QR codes\Benign\benign'

# List to store the labels ('malicious' or 'non-malicious') and images
labels = []
images = []

# Load malicious images
for filename in os.listdir(malicious_dir):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        labels.append('malicious')
        img = cv2.imread(os.path.join(malicious_dir, filename), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128, 128))
        images.append(img)

# Load non-malicious images
for filename in os.listdir(non_malicious_dir):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        labels.append('non-malicious')
        img = cv2.imread(os.path.join(non_malicious_dir, filename), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128, 128))
        images.append(img)

# Convert labels to numerical values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
onehot_encoder = OneHotEncoder(sparse=False)
labels_encoded = onehot_encoder.fit_transform(labels_encoded.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42)
X_train = np.array(X_train) / 255.0
X_test = np.array(X_test) / 255.0

# Reshape the input for LSTM model
X_train = np.reshape(X_train, (X_train.shape[0], 128, 128))
X_test = np.reshape(X_test, (X_test.shape[0], 128, 128))

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(128, 128), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Now you can use the trained model to make predictions on new data
# Make sure to preprocess the new data similarly before making predictions


c:\Users\shlok\anaconda3\envs\MachineLearning\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
100/100 [==============================] - 45s 344ms/step - loss: 0.5812 - accuracy: 0.7000 - val_loss: 0.4662 - val_accuracy: 0.7812
Epoch 2/10
100/100 [==============================] - 32s 323ms/step - loss: 0.3791 - accuracy: 0.8441 - val_loss: 0.4777 - val_accuracy: 0.7937
Epoch 3/10
100/100 [==============================] - 32s 316ms/step - loss: 0.2523 - accuracy: 0.9003 - val_loss: 0.2425 - val_accuracy: 0.9075
Epoch 4/10
100/100 [==============================] - 32s 322ms/step - loss: 0.1960 - accuracy: 0.9222 - val_loss: 0.2399 - val_accuracy: 0.9100
Epoch 5/10
100/100 [==============================] - 32s 320ms/step - loss: 0.1776 - accuracy: 0.9334 - val_loss: 0.2057 - val_accuracy: 0.9200
Epoch 6/10
100/100 [==============================] - 32s 316ms/step - loss: 0.1367 - accuracy: 0.9469 - val_loss: 0.1865 - val_accuracy: 0.9250
Epoch 7/10
100/100 [==============================] - 31s 312ms/step - loss: 0.1077 - accuracy: 0.9622 - val_loss: 0.1227 - val_ac

In [39]:
model.save('qr_code_BinaryLSTM_model.h5')

In [3]:
from keras.models import load_model 
import cv2
import numpy as np
model = load_model('qr_code_BinaryLSTM_model.h5')
def classify_qr_code(qr_code_image_path):
    qr_code_image = cv2.imread(qr_code_image_path)
    gray_image = cv2.cvtColor(qr_code_image, cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray_image, (128, 128))
    qr_code_data = np.expand_dims(resized_image / 255.0, axis=0)
    prediction = model.predict(qr_code_data)
    label = np.argmax(prediction)
    if label == 0:
        return 'MALICIOUS'
    else:
        return 'NOT MALICIOUS'
# Example usage
qr_code_image =r'C:\Users\ACER\OneDrive\Desktop\Final Project\QR codes\Malicious\malicious\malicious_416236.png'
classification_result = classify_qr_code(qr_code_image)
print('Classification result:', classification_result)

NameError: name 'np' is not defined

In [5]:
len(y_test)

800

In [26]:
output=model.predict(X_test)

25/25 [==============================] - 3s 132ms/step


In [22]:
y_pred=[]
for i in output:
     y_pred.append(np.argmax(i))
y_pred=np.array(y_pred)

In [31]:
new_y_test=[]
for i in y_test:
    new_y_test.append(np.argmax(i))
    

In [6]:
from sklearn.metrics import confusion_matrix,f1_score,precision_recall_curve,accuracy_score,roc_auc_score,precision_recall_curve
confusion_matrix(new_y_test,y_pred)

NameError: name 'new_y_test' is not defined

In [35]:
accuracy_score(new_y_test,y_pred)

0.96

In [36]:
f1_score(new_y_test,y_pred)

0.9571045576407506

In [38]:
roc_auc_score(new_y_test,y_pred)

0.959189046866772

In [5]:
precision, recall, _ = precision_recall_curve(new_y_test,y_pred)
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

NameError: name 'precision_recall_curve' is not defined